Comparing Phoenix models with G430L

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
import os
import glob
from astropy.table import Table
from astropy.io import ascii
import astropy.units as u
import astropy.constants as const
from scipy.interpolate import interpolate
from craftroom import resample
from astropy.convolution import convolve, Box1DKernel
from astropy.modeling import models, fitting
from scipy.optimize import leastsq

#matplotlib set up
%matplotlib inline
from matplotlib import rcParams
rcParams["figure.figsize"] = (14, 5)
rcParams["font.size"] = 20


In [2]:
path = '/media/david/5tb_storage1/muscles/stis_x1ds/'
allspecs = np.hstack((glob.glob(path+'*x1d.fits'), glob.glob(path+'*sx1.fits')))
#allspecs
stars = os.listdir(path)
stars


FileNotFoundError: [Errno 2] No such file or directory: '/media/david/5tb_storage1/muscles/stis_x1ds/'

In [ ]:
from astropy.convolution import convolve_fft
from astropy.convolution import Gaussian1DKernel

def smear(w,f, R, w_sample=1):
    '''
    Smears a model spectrum with a gaussian kernel to the given resolution, R.
    Adapeted from https://github.com/spacetelescope/pysynphot/issues/78

    Parameters
    -----------

    w,f:  spectrum to smear

    R: int
        The resolution (dL/L) to smear to

    w_sample: int
        Oversampling factor for smoothing

    Returns
    -----------

    sp: PySynphot Source Spectrum
        The smeared spectrum
    '''

    # Save original wavelength grid and units
    w_grid = w
    

    # Generate logarithmic wavelength grid for smoothing
    w_logmin = np.log10(np.nanmin(w_grid))
    w_logmax = np.log10(np.nanmax(w_grid))
    n_w = np.size(w_grid)*w_sample
    w_log = np.logspace(w_logmin, w_logmax, num=n_w)

    # Find stddev of Gaussian kernel for smoothing
    R_grid = (w_log[1:-1]+w_log[0:-2])/(w_log[1:-1]-w_log[0:-2])/2
    sigma = np.median(R_grid)/R
    if sigma < 1:
        sigma = 1

    # Interpolate on logarithmic grid
    f_log = np.interp(w_log, w_grid, f)

    # Smooth convolving with Gaussian kernel
    gauss = Gaussian1DKernel(stddev=sigma)
    f_conv = convolve_fft(f_log, gauss)

    # Interpolate back on original wavelength grid
    f_sm = np.interp(w_grid, w_log, f_conv)

    # Write smoothed spectrum back into Spectrum object
    return w_grid, f_sm

In [ ]:
phoenix_path = '/media/david/5tb_storage1/muscles/phoenix_models/'
g430ls = []
grating_order = ['G430L','G230L', 'G230LB']
for star in stars[1:]:
    print(star)
    specs = glob.glob('../common/stis_test_output/{}/*ecsv'.format(star))
    w1 = 2500
    for grating in grating_order:
        for spec in specs:
            data= Table.read(spec)
            if data.meta['GRATING'] == grating:
                w, f, e = data['WAVELENGTH'], data['FLUX'], data['ERROR']
                mask = (w > w1)
                w, f, e = w[mask], f[mask], e[mask]
                plt.step(w, f, where='mid')
                if grating == 'G430L':
                    g430ls.append(spec)
                    print(spec)
                    bin_width = 30
                    sn = np.array([np.mean(f[i:i+bin_width]/e[i:i+bin_width]) for i in range(len(w[:-bin_width]))])
                    start = w[:-bin_width][np.where(sn > 1)[0][0]]
                    plt.axvline(start, ls='--', c='C2')
                    plt.step(w[w >start], f[w > start], where='mid')

                    
                    plt.step(w, e, where='mid')
                    #emask = (f/e >1)
                    #plt.step(w[emask], f[emask], where='mid')
                #w1 = w[-1]
            
    opath = glob.glob(phoenix_path+star+'*')
   # print(opath)
    pdata = Table.read(opath[0], data_end= 200000)
    w, f, e  = pdata['WAVELENGTH'], pdata['FLUX']*pdata.meta['NORMFAC'], np.zeros(len(pdata['WAVELENGTH']))
    mask = w > 2500
    w, f, e = w[mask], f[mask], e[mask]
    w, f = smear(w, f, 500)
    plt.plot(w, f, ls='--')
    #plt.xscale('log')
    plt.yscale('log')
    plt.ylim(1e-17)
    plt.show()
        

In [ ]:

#Table.read.help('ascii.ecsv')
testspec = '../common/stis_test_output/GJ1132/hlsp_muscles_hst_stis_gj1132_g430l_v1_component-spec.ecsv'
data = Table.read(testspec)
w, f, e = data['WAVELENGTH'], data['FLUX'], data['ERROR']
bin_width = 30
sn = np.array([np.mean(f[i:i+bin_width]/e[i:i+bin_width]) for i in range(len(w[:-bin_width]))])
plt.plot(w[:-bin_width], sn)
plt.axhline(1, ls='--', c='C1')
start = w[:-50][np.where(sn > 1)[0][0]]
plt.axvline(start, ls='--', c='C2')
plt.show()

plt.plot(w,f)
plt.plot(w,e)
plt.yscale('log')

plt.step(w[w >start], f[w > start], where='mid')


In [ ]:
for spec in g430ls:
    data = Table.read(spec)
    w, f, e = data['WAVELENGTH'], data['FLUX'], data['ERROR']
    widths = np.arange(5, 100, 5)
    cut_starts = []
    for bin_width in widths:
        sn = np.array([np.mean(f[i:i+bin_width]/e[i:i+bin_width]) for i in range(len(w[:-bin_width]))])
        start = w[:-bin_width][np.where(sn > 1)[0][0]]
        cut_starts.append(start)
    plt.plot(widths, cut_starts)

In [ ]:
from matplotlib.gridspec import GridSpec
import matplotlib.patches as patches
ca =[ 3933.6614, 3968.4673]

def residuals(scale, f, mf):
    return f - mf/scale

grating_order = ['G430L']
for star in stars[1:]:
    fig = plt.figure(figsize=(14, 8))
    gs = GridSpec(6, 6, figure=fig)
    ax = plt.subplot(gs[:4, :-2])
    print(star)
    specs = glob.glob('../common/stis_test_output/{}/*ecsv'.format(star))
    for grating in grating_order:
        for spec in specs:
            data= Table.read(spec)
            if data.meta['GRATING'] == grating:
                w, f, e = data['WAVELENGTH'], data['FLUX'], data['ERROR']
                bin_width = 30
                sn = np.array([np.mean(f[i:i+bin_width]/e[i:i+bin_width]) for i in range(len(w[:-bin_width]))])
                start = w[:-bin_width][np.where(sn > 1)[0][0]]
                mask = (w > start) & (f > 0)
                w1, f1 , e1 = w[mask], f[mask], e[mask]
                plt.step(w1, f1, where='mid', label=grating)
                
               
    opath = glob.glob(phoenix_path+star+'*')
    pdata = Table.read(opath[0], data_end= 200000)
    mw, mf = pdata['WAVELENGTH'], pdata['FLUX']*pdata.meta['NORMFAC']
    print(pdata.meta['NORMFAC'])
    mask = (mw >= w[0]) & (mw <=w[-1] ) 
    mw, mf = mw[mask], mf[mask]
    mw, mf = smear(mw, mf, 1000)
    plt.plot(mw, mf, ls='--', label='PHOENIX')
    plt.legend()
    #plt.xscale('log')
    plt.yscale('log')
    plt.ylabel('Flux (erg s$^{-1}$ cm$^{-2}$ \AA$^{-1}$)', size=20)

#plt.xlabel('Wavelength (\AA)', size=20)
#plt.xticks(visible=False)
    pfr = interpolate.interp1d(mw, mf, fill_value='extrapolate')(w1)
    scale, flag = leastsq(residuals, 1., args=(f1[w1 > 4000], pfr[w1> 4000]))
  #  plt.plot(w1, f1*scale)
    print(scale)
    
  
  
    plt.subplot(gs[4:,:-2])
    #pwo, pfr = resample.bintogrid(mw, mf, newx=w)
     #print(len(w), len(pwo), len(pfr))
    plt.plot(w1, f1/pfr)
    
    #print(np.median(f1/pfr))
    plt.xlabel('Wavelength (\AA)', size=20)
    plt.ylabel('F$_{\mathrm{G430L}}$/F$_{\mathrm{PHX}}$')
   # plt.xlim(wo[0]-10, wo[-1]+10)
    plt.axhline(1.0, ls='--', c='0.7')
    plt.yscale('log')

    plt.subplot(gs[:4,-2:])
    mask = (w > 3850) & (w < 4050)
    plt.step(w[mask],  f[mask], where='mid')
    #plt.errorbar(w[::3], f[::3], yerr=e[::3], c='C0', ls='none', capsize=5, label = 'G430L')

    [plt.axvline(line, c='r') for line in ca]
   # [plt.annotate('',(line, 1.23e-16), xytext=(line, 1.33e-16), horizontalalignment='center', arrowprops=dict(arrowstyle='-')) for line in ca]
   # plt.annotate('Ca\,{\sc ii}',(np.mean(ca), 6e-17), xytext=(np.mean(ca), 1.35e-16), horizontalalignment='center')

    plt.xlim(3851,4049)
 #   plt.ylim(0.1e-16, 1.49e-16)
#    plt.plot(wo[mask], gg_fit(wo[mask]), lw=2, label='Model fit')
    #plt.legend()
   # plt.xlabel('Wavelength (\AA)', size=20)
    
    plt.subplot(gs[4:,-2:])
    scales = []
    wcut = []
    for wi in np.arange(3000, 5501, 100):
        if wi > w1[0]:
            mask = (w1 > wi)
            scales.append(leastsq(residuals, 1., args=(f1[mask], pfr[mask]))[0])
            wcut.append(wi)
    plt.plot(wcut, scales)        
    plt.xlim(2900, 5600)
    plt.xlabel('Wavelength (\AA)', size=20)
    
    plt.tight_layout()
    plt.subplots_adjust(hspace=0.45)
    plt.show()


In [ ]:
plt.plot(mw, mf)
plt.xlim(4200, 4250)
plt.ylim(0, 2e-15)

Scaling we can come back to. In most cases, the noisy bit of the G430L spectrum is adequately covered by the NUV spectrum. What about X-shooter?

20210122 making new versions for the MM meeting

In [ ]:
stars

In [ ]:
casleo=  glob.glob('../../casleo/*.txt')
print(casleo)
casnums = [os.path.split(cas)[1][2:5] for cas in casleo]
casnums

In [ ]:
for star in stars:
    print(star)

In [ ]:
#from matplotlib.gridspec import GridSpec
#import matplotlib.patches as patches
ca =[ 3933.6614, 3968.4673]

xpath = '../../xshooter/archive/'

def residuals(scale, f, mf):
    return f - mf/scale

def normpoint(w, f, wn): #normalise around a point wn
    mask = (w < wn-5) & (w > wn+5)
    fn = np.mean(f[mask])
    return fn

grating_order = ['G430L']
for star in stars[1:]:
    fig = plt.figure(figsize=(14, 8))
    gs = GridSpec(6,1, figure=fig)
    ax = plt.subplot(gs[:4])
    print(star)
    specs = glob.glob('../common/stis_test_output/{}/*ecsv'.format(star))
    for grating in grating_order:
        for spec in specs:
            data= Table.read(spec)
            if data.meta['GRATING'] == grating:
                w, f, e = data['WAVELENGTH'], data['FLUX'], data['ERROR']
                bin_width = 30
                sn = np.array([np.mean(f[i:i+bin_width]/e[i:i+bin_width]) for i in range(len(w[:-bin_width]))])
                start = w[:-bin_width][np.where(sn > 1)[0][0]]
                mask = (w > 3800) & (f > 0)
                w1, f1 , e1 = w[mask], f[mask], e[mask]
                plt.step(w1, f1, where='mid', label=grating, c='C0', alpha=0.9)
    
                
               
    opath = glob.glob(phoenix_path+star+'*')
    pdata = Table.read(opath[0], data_end= 200000)
    mw, mf = pdata['WAVELENGTH'], pdata['FLUX']*pdata.meta['NORMFAC']
    print(pdata.meta['NORMFAC'])
    mask = (mw >= 3300) & (mw <=w[-1] ) 
    mw, mf = mw[mask], mf[mask]
    mw, mf = smear(mw, mf, 1000)
    plt.plot(mw, mf, ls='--', label='PHOENIX', c='C1')
    
    if star[2:5] in casnums:
        caspath = glob.glob('../../casleo/*{}*.txt'.format(star[2:5]))
        print(caspath)
        wc, fc = np.loadtxt(caspath[0], unpack=True)
        wc, fc = wc[fc < 1e-12], fc[fc < 1e-12] #one star has a dodgy start
        wc, fc = smear(wc, fc, 1000)
        plt.plot(wc, fc, zorder=-10, label='CASLEO', c='C2', alpha=0.9)
    
    xfiles = glob.glob('{}{}*UVB.dat'.format(xpath, star))
    print(xfiles)
    if len(xfiles) > 0:
        xw, xf, xe = np.loadtxt(xfiles[0], unpack=True)
        xw *=10
        xmask = (xf > 0) & (xw > 3300) 
        xw, xf = xw[xmask], xf[xmask]
        xw, xf = smear(xw, xf, 1000)
        plt.plot(xw, xf, c='C3', label='X-shooter', zorder=-10, alpha=0.9)
    
    
    plt.legend(loc=4)
    #plt.xscale('log')
    plt.yscale('log')
    plt.ylabel('Flux (erg s$^{-1}$ cm$^{-2}$ \AA$^{-1}$)', size=25)

#plt.xlabel('Wavelength (\AA)', size=20)
    plt.xticks(visible=False)
    pfr = interpolate.interp1d(mw, mf, fill_value='extrapolate')(w1)
    scale, flag = leastsq(residuals, 1., args=(f1[w1 > 4000], pfr[w1> 4000]))
  #  plt.plot(w1, f1*scale)
    print(scale)
    plt.xlim(3200, 5900)
#     plt.ylim(1.1e-15)

    plt.title(star)
  
    plt.subplot(gs[4:])
    #pwo, pfr = resample.bintogrid(mw, mf, newx=w)
     #print(len(w), len(pwo), len(pfr))
    plt.plot(w1, f1/pfr)
    
    #print(np.median(f1/pfr))
    plt.xlabel('Wavelength (\AA)', size=25)
    plt.ylabel('F$_{\mathrm{G430L}}$/F$_{\mathrm{PHX}}$', size=25)
   # plt.xlim(wo[0]-10, wo[-1]+10)
    plt.axhline(1.0, ls='--', c='0.7')
    plt.yscale('log')
    plt.xlim(3200, 5900)
    plt.ylim(1.3e-1, 0.7e1)
    
 
    plt.tight_layout()
    plt.subplots_adjust(hspace=0.01)
    plt.savefig('comp_plots/{}_optical.pdf'.format(star), dpi=300)
    plt.savefig('comp_plots/{}_optical.png'.format(star), dpi=300)
    plt.show()


In [ ]:
caspath